In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import autograd
import matplotlib.pyplot as plt
import random
import math
import time

In [2]:
# Define a small network with inputs of dimension 5
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(2, 10)
        self.rel = nn.ReLU()
        self.linear2 = nn.Linear(10, 1)

    def forward(self, x, **args):
        x = self.linear1(x)
        x = self.rel(x)
        x = self.linear2(x)
        return x

In [3]:
def reshape_jacobian(jac):
    # this function is useful for post-processing the output of `torch.autograd.functional.jacobian`. The issue is that if we input
    # a tensor of shape (m,n), torch.autograd.functional.jacobian outputs a tensor of shape (m, k, m, n) where k is the dimension of
    # the output of the neural network.
    dim1 = jac.shape[0]
    new_jac = torch.clone(jac[0, :, :, :])
    for i in range(0, dim1):
        # the (real) jacobian of the j-th output at input i is jac[i, j]
        new_jac[:, i, :] = jac[i, :, i, :]
    if jac.shape[1] == 1:
        # if the output has dimension 1 (as in the case of our model) the kill the first dimension.
        new_jac = new_jac[0, :, :]
    return new_jac


def get_jacobian(model, x, scale):
    # this corresponds to the original way of computing the gradient
    x_input = autograd.Variable(x, requires_grad=True)
    output = model.forward(x_input)
    grad = (
        torch.round(
            scale
            * autograd.grad(output[:, 0].sum(), x_input, retain_graph=True)[0].data
        )
        / scale
    )
    return grad


def get_region(model, x, scale=1e8, use_torch_jacobian=False):
    # This function returns a list giving the region corresponding to each input in x, and the total number of regions.

    # compute jacobian at each point in input tensor x
    if use_torch_jacobian:
        grad = torch.autograd.functional.jacobian(model.forward, x, vectorize=True)
        grad = reshape_jacobian(grad)
    else:
        grad = get_jacobian(model, x, scale)

    # remove repetitions from grad (along dimension 0, i.e. repetitions of jacobians)
    region_grad = torch.unique(grad, dim=0)
    # initialise 1D tensor that will contain the index of the region corresponding to each point in `x`
    region = torch.zeros(x.shape[0], dtype=int)

    # now we iterate through all the jacobians attained in region_grad. At each step we will compute the indices of the
    # inputs in x which attained a specific jacobian and update the `region` tensor accordingly.
    for index, r in enumerate(region_grad):
        # write the gradient as a tensor of shape (m, 2) where m is the number of points
        reshaped_grad = grad.reshape(x.shape[0], -1)
        # compute the 2D tensor whose ij-th entry is True if reshaped_grad[i,j] = r[j]
        test_eq = torch.eq(reshaped_grad, r)
        # take the sum over columns to get the 1D tensor whose i-th entry is 2 precisely when reshaped_grad[i,:] = r
        test_eq = torch.sum(test_eq, axis=1)
        # get the indices where this equality holds
        l = torch.where(test_eq == 2)
        # set the entry of region at each one of these indices to be `index`
        region[l] = index
    return (region, region_grad.shape[0])

In [4]:
my_nn = Net()
GRID_SIZE = 100
x = torch.linspace(-1, 1, GRID_SIZE)
y = torch.linspace(-1, 1, GRID_SIZE)
X, Y = torch.meshgrid(x, y)
# write the meshgrid as a tensor of shape (GRID_SIZE**2, 2)
XY = torch.stack((X, Y), axis=2).reshape(-1, 2)
R, n_reg = get_region(my_nn, XY)

print("Total number of regions is ", n_reg)

Total number of regions is  23


/Users/shivbhatia/AllFiles/dev/tropical_nn_python/.venv/lib/python3.10/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3588.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
## autograd.jacobian vs the original method:

start = time.time()
R, n_reg = get_region(my_nn, XY)
mid = time.time()
R, n_reg = get_region(my_nn, XY, use_torch_jacobian=True)
end = time.time()

print("Initial method took ", mid - start, " seconds")
print("Autograd Jacobian method took", end - mid, " seconds")

Initial method took  0.01823115348815918  seconds
Autograd Jacobian method took 2.6742889881134033  seconds
